In [131]:
from glob import iglob as list_paths


def load_text(path):
    with open(path, encoding='utf-8') as file:
        return file.read()


texts = [
    load_text(_)
    for _ in list_paths('3_fav_ex/*.txt')
]

In [132]:
from natasha import (
    NamesExtractor,
    SimpleNamesExtractor,
    PersonExtractor,

    LocationExtractor,
    AddressExtractor,

    OrganisationExtractor,

    DatesExtractor,

    MoneyExtractor,
    MoneyRateExtractor,
    MoneyRangeExtractor,
)

from natasha.markup import (
    show_markup_notebook as show_markup,
    format_json
)

In [133]:
from yargy.token import Span

def unite_spans(spans):
    if not spans:
        return spans
    spans = sorted(spans, key=lambda x: (x.start, x.stop))
    new_spans = []
    beg, end = spans[0].start, spans[0].stop
#     prev = spans[0]
    for span in spans[1:]:
        if span.start < end < span.stop:
            end = span.stop
        elif span.stop <= end:
            continue
        else:
            new_spans.append(Span(beg, end))
            beg, end = span.start, span.stop
    new_spans.append(Span(beg, end))
    return new_spans

In [134]:
def find_entities(text):
    all_spans = []
    for extr_cls in (
        NamesExtractor,
#         SimpleNamesExtractor,
#         PersonExtractor,
#         LocationExtractor,
        AddressExtractor,
#         OrganisationExtractor,
        DatesExtractor,
#         MoneyExtractor,
#         MoneyRateExtractor,
#         MoneyRangeExtractor,
    ):
        extractor = extr_cls()
        matches = extractor(text)
        spans = [_.span for _ in matches]
        all_spans.extend(spans)
    return unite_spans(all_spans)


for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)
    print('\n\n\n')

In [135]:
def find_entities(text):
    all_spans = []
    for extr_cls in (
        NamesExtractor,
#         SimpleNamesExtractor,
#         PersonExtractor,
#         LocationExtractor,
        AddressExtractor,
#         OrganisationExtractor,
        DatesExtractor,
#         MoneyExtractor,
#         MoneyRateExtractor,
#         MoneyRangeExtractor,
    ):
        extractor = extr_cls()
        matches = extractor(text)
        spans = [_.span for _ in matches]
        all_spans.extend(spans)
    return unite_spans(all_spans)


for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)
    print('\n\n\n')

In [136]:
texts = ['Здравствуйте, Алексей Сергеевич',
         'Здравствуйте, Алексей Сергеевич!',
         'Здравствуйте, Максим Сергеевич!',
         'Санкт-Петербург',
        'наб. Крюкова канала, д. 12, Санкт-Петербург.',
        'Санкт-Петербург, наб. Крюкова канала, д. 12']

for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)

In [137]:
texts = '''
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими).
Предлагаю вернуть прежние границы природного парка №71 на Инженерной улице 2. По адресу Алтуфьевское шоссе д.51 (основной вид разрешенного использования: производственная деятельность, склады) размещен МПЗ. Жители требуют незамедлительной остановки МПЗ и его вывода из района
Контакты О нас телефон 7 881 574-10-02 Адрес Республика Карелия,г.Петрозаводск,ул.Маршала Мерецкова, д.8 Б,офис 4 
Благодарственное письмо Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника: Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. Они вовлекали сына в интересные внеурочные занятия, связанные с театром и походами.
'''.splitlines()

for text in texts:
    all_spans = find_entities(text)
    show_markup(text, all_spans)

In [138]:
def human_friendly_print(path_exp=None, line_len=5, word_width=20):
    if path_exp is None:
        path_exp = input('path to text files:')
    for path in list_paths(path_exp):
        print()
        print(path)
        text = load_text(path)
        words = text.split()
        t = []
        print()
        for i, word in enumerate(words):
            t.append((i, word))
            if len(t) == line_len or i == len(words) - 1:
                print(''.join([x[1].ljust(word_width) for x in t]))
                print(''.join([str(x[0]).ljust(word_width) for x in t]))
                print()
                t = []

human_friendly_print('3_fav_ex/*.txt')


3_fav_ex/let1.txt

Уважаемый           Максим              Сергеевич           Рассказов!          Приглашаем          
0                   1                   2                   3                   4                   

Вас                 посмотреть          в                   кругу               друзей              
5                   6                   7                   8                   9                   

и                   знакомых            новый,              научно-популярный   фильм               
10                  11                  12                  13                  14                  

под                 названием           «Изобретение        штанов!».           Изобретение         
15                  16                  17                  18                  19                  

штанов?             Мы                  совсем              не                  задумываемся        
20                  21                  22                  23     

In [139]:
import random
import string


def find_word_borders(text):
    borders = []
    words = text.split()
    start = 0
    for word in words:
        word_beg = text.find(word, start)
        start = word_end = word_beg + len(word)
        borders.append((word_beg, word_end))
    return borders


def inds_to_spans(text, inds):
    inds.sort()
    spans = []
    borders = find_word_borders(text)
    for ind in inds:
        spans.append(borders[ind])
    return spans


def spans_to_inds(text, spans):
    spans.sort()
    inds = []
    borders = find_word_borders(text)
    cur_ind = 0
    for span in spans:
        while cur_ind < len(borders) and borders[cur_ind][1] <= span[0]:
            cur_ind += 1
        while cur_ind < len(borders) and borders[cur_ind][0] < span[1]:
            inds.append(cur_ind)
            cur_ind += 1
    return inds


def test(max_word=15, max_dict=20_000, max_text=100, highlight_share=0.1, n_iter=10_000):
    words = {''.join(random.choices(string.ascii_lowercase, k=random.randrange(1, max_word))) for _ in range(max_dict)}
    print(len(words))
    for _ in range(n_iter):
        text_len = random.randrange(1, max_text)
        word_seq = random.choices(list(words), k=text_len)
        text = ' '.join(word_seq)
        inds = random.sample(range(text_len), k=int(highlight_share * text_len))
        assert inds == spans_to_inds(text, inds_to_spans(text, inds)), (inds, spans_to_inds(text, inds_to_spans(text, inds)))
    print('test ok')

test()

17710
test ok


In [140]:
def precision(inds_true, inds_pred):
    if not inds_pred:
        return None
    return len(set(inds_true) & set(inds_pred)) / len(set(inds_pred))


def recall(inds_true, inds_pred):
    if not inds_true:
        return None
    return len(set(inds_true) & set(inds_pred)) / len(set(inds_true))


def f1(inds_true, inds_pred):
    pr, rec = precision(inds_true, inds_pred), recall(inds_true, inds_pred)
    if pr is None or rec is None:
        return None
    if pr + rec == 0:
        return 0
    return 2 * pr * rec / (pr + rec)
    

In [141]:
precision([2, 4, 6, 8], [2, 4, 6, 1, 24]), recall([2, 4, 6, 8], [2, 4, 6, 1, 24]), f1([2, 4, 6, 8], [2, 4, 6, 1, 24]), 

(0.6, 0.75, 0.6666666666666665)

In [142]:
from yargy.token import Span

summary_inds, texts = [], []
words = {''.join(random.choices(string.ascii_lowercase, k=random.randrange(1, 15))) for _ in range(20_000)}
# print(len(words))
for _ in range(3):
    text_len = random.randrange(10, 100)
    word_seq = random.choices(list(words), k=text_len)
    text = ' '.join(word_seq)
    inds = random.sample(range(text_len), k=int(0.1 * text_len))
    texts.append(text)
    summary_inds.append(inds)

all_spans = []
for inds, text in zip(summary_inds, texts):
    spans = []
    borders = []
    words = text.split()
    start = 0
    for word in words:
        word_beg = text.find(word, start)
        start = word_end = word_beg + len(word)
        borders.append((word_beg, word_end))
    for ind in inds:
        spans.append(Span(*borders[ind]))
    show_markup(text, spans)

In [143]:
import json

def load_inds(path):
    with open(path) as f:
        inds = json.loads(f.read())
    return inds

texts = [
    load_text(_)
    for _ in list_paths('3_fav_ex/*.txt')
]

summary_inds = [
    load_inds(path)
    for path in list_paths('3_fav_ex/*sum.json')
]

In [144]:
for inds, text in zip(summary_inds, texts):
    spans = inds_to_spans(text, inds)
    show_markup(text, [Span(*s) for s in spans])


In [149]:
texts = [
    load_text(_)
    for _ in list_paths('3_fav_ex/*.txt')
]

all_true_inds = [
    load_inds(path)
    for path in list_paths('3_fav_ex/*sum.json')
]

print(*(str(x).ljust(20) for x in ('path', 'precision', 'recall', 'f1')), sep='|')
for path in list_paths('3_fav_ex/*.txt'):
    text = load_text(path)
    sum_path = path.replace('.txt', '_sum.json')
    true_inds = load_inds(sum_path)
    spans = find_entities(text)
    inds = spans_to_inds(text, spans)
    pr = precision(true_inds, inds)
    rec = recall(true_inds, inds)
    f1_ = f1(true_inds, inds)
    print(*(str(x).ljust(20) for x in (path, pr, rec, f1_)), sep='|')

path                |precision           |recall              |f1                  
3_fav_ex/let1.txt   |1.0                 |0.30434782608695654 |0.4666666666666667  
3_fav_ex/let2.txt   |0.0                 |0.0                 |0                   
3_fav_ex/let3.txt   |0.4                 |0.1                 |0.16000000000000003 
3_fav_ex/let4.txt   |0.0                 |0.0                 |0                   
3_fav_ex/news1.txt  |1.0                 |0.09523809523809523 |0.17391304347826084 
3_fav_ex/post1.txt  |None                |0.0                 |None                
3_fav_ex/post2.txt  |1.0                 |0.17391304347826086 |0.29629629629629634 
